# Notebook for cleaning `PATIENTS TABLE`

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text

In [2]:
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")

In [3]:
def sql(query , engine = engine):
    return pd.read_sql(query,engine)

In [4]:
sql('select * from patients limit 5')

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,None,999-76-6866,S99984236,X19277260X,Mr.,José Eduardo181,Gómez206,None,...,Marigot Saint Andrew Parish DM,427 Balistreri Way Unit 19,Chicopee,Massachusetts,Hampden County,1013.0,42.228354,-72.562951,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,None,999-73-5361,S99962402,X88275464X,Mr.,Milo271,Feil794,None,...,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,None,999-27-3385,S99972682,X73754411X,Mr.,Jayson808,Fadel536,None,...,Springfield Massachusetts US,1056 Harris Lane Suite 70,Chicopee,Massachusetts,Hampden County,1020.0,42.181642,-72.608842,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,None,999-85-4926,S99974448,X40915583X,Mrs.,Mariana775,Rutherford999,None,...,Yarmouth Massachusetts US,999 Kuhn Forge,Lowell,Massachusetts,Middlesex County,1851.0,42.636143,-71.343255,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,None,999-60-7372,S99915787,X86772962X,Mr.,Gregorio366,Auer97,None,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,598763.07,3772.20


In [5]:
query = '''
    select column_name,data_type from information_schema.columns
    where table_name = 'patients' and
    table_schema = database();
'''
sql(query)

,COLUMN_NAME,DATA_TYPE
0,Id,text
1,BIRTHDATE,text
2,DEATHDATE,text
3,SSN,text
4,DRIVERS,text
5,PASSPORT,text
6,PREFIX,text
7,FIRST,text
8,LAST,text
9,SUFFIX,text


In [6]:
query = '''
    alter table patients
    drop column deathdate,
    drop column ssn,
    drop column passport,
    drop column prefix,
    drop column first,
    drop column last,
    drop column suffix,
    drop column maiden,
    drop column birthplace,
    drop column address,
    drop column zip,
    drop column lat,
    drop column lon;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

In [8]:
query = '''
    alter table patients
    rename column HEALTHCARE_EXPENSES to EXPENSES,
    rename column HEALTHCARE_COVERAGE to COVERAGE;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

In [9]:
sql('select * from patients limit 5')

,Id,BIRTHDATE,DRIVERS,MARITAL,RACE,ETHNICITY,GENDER,CITY,STATE,COUNTY,EXPENSES,COVERAGE
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,S99984236,M,white,hispanic,M,Chicopee,Massachusetts,Hampden County,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,S99962402,M,white,nonhispanic,M,Somerville,Massachusetts,Middlesex County,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,S99972682,M,white,nonhispanic,M,Chicopee,Massachusetts,Hampden County,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,S99974448,M,white,nonhispanic,F,Lowell,Massachusetts,Middlesex County,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,S99915787,None,white,nonhispanic,M,Boston,Massachusetts,Suffolk County,598763.07,3772.20


In [10]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'patients' and
    table_schema = database();
'''
sql(query)
patient_columns = [c[0] for c in sql(query).values.tolist()]

In [11]:
for col in patient_columns:
    query = f'''
        alter table patients
        rename column {col} to {col.lower()}
    '''
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [12]:
sql('select * from patients limit 5')

,id,birthdate,drivers,marital,race,ethnicity,gender,city,state,county,expenses,coverage
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,S99984236,M,white,hispanic,M,Chicopee,Massachusetts,Hampden County,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,S99962402,M,white,nonhispanic,M,Somerville,Massachusetts,Middlesex County,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,S99972682,M,white,nonhispanic,M,Chicopee,Massachusetts,Hampden County,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,S99974448,M,white,nonhispanic,F,Lowell,Massachusetts,Middlesex County,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,S99915787,None,white,nonhispanic,M,Boston,Massachusetts,Suffolk County,598763.07,3772.20


In [19]:
for col in patient_columns:
    query = f'''
        select sum({col} is null) as {col}_null from patients
        having {col}_null > 0;
    '''
    display(sql(query))

,Id_null


,BIRTHDATE_null


,DRIVERS_null
0,213.0


,MARITAL_null
0,380.0


,RACE_null


...

,CITY_null


,STATE_null


,COUNTY_null


,EXPENSES_null


,COVERAGE_null


In [20]:
query = '''
    update patients
    set marital = 'u' where marital is null;
'''
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [21]:
query = '''
    alter table patients
    drop column drivers;
'''
with engine.connect() as con:
    con.execute(text(query))
    con.commit()

In [22]:
query = """
UPDATE patients
SET birthdate = STR_TO_DATE(
    TRIM(REPLACE(REPLACE(REPLACE(birthdate, '"', ''), '\r', ''), '\n', '')),
    '%Y-%m-%d'
);
"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [23]:
query = """
ALTER TABLE patients
MODIFY birthdate DATE;
"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [24]:
sql('select * from patients limit 5')

,id,birthdate,marital,race,ethnicity,gender,city,state,county,expenses,coverage
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,M,white,hispanic,M,Chicopee,Massachusetts,Hampden County,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,M,white,nonhispanic,M,Somerville,Massachusetts,Middlesex County,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,M,white,nonhispanic,M,Chicopee,Massachusetts,Hampden County,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,M,white,nonhispanic,F,Lowell,Massachusetts,Middlesex County,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,u,white,nonhispanic,M,Boston,Massachusetts,Suffolk County,598763.07,3772.20


In [26]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'patients' and
    table_schema = database();
'''
sql(query)
patient_columns = [c[0] for c in sql(query).values.tolist()]

In [27]:
for col in patient_columns:
    query = f'''
        update patients
        set {col} = lower({col})
    '''
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [28]:
sql('select * from patients limit 5')

,id,birthdate,marital,race,ethnicity,gender,city,state,county,expenses,coverage
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,m,white,hispanic,m,chicopee,massachusetts,hampden county,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,m,white,nonhispanic,m,somerville,massachusetts,middlesex county,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,m,white,nonhispanic,m,chicopee,massachusetts,hampden county,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,m,white,nonhispanic,f,lowell,massachusetts,middlesex county,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,u,white,nonhispanic,m,boston,massachusetts,suffolk county,598763.07,3772.20
